In [33]:
# Import dataset from Kaggle
import kagglehub
arnavs19_underwater_plastic_pollution_detection_path = kagglehub.dataset_download('arnavs19/underwater-plastic-pollution-detection')

print('Data source import complete.')


Data source import complete.


In [34]:
# Download latest version
path = kagglehub.dataset_download("arnavs19/underwater-plastic-pollution-detection")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/arnavs19/underwater-plastic-pollution-detection/versions/1


In [35]:
#read from the path
from pathlib import Path
path = Path('/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics')
# Labels of first 10 training images
real_files = list((path/'train/images').glob("*.jpg"))
real_files[:10]

[PosixPath('/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/train/images/uwg_yt-40__fliph_jpg.rf.f91d5494c350a9fae0dc5cc91d8e7dee.jpg'),
 PosixPath('/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/train/images/1601024846172-gettyimagkjhes-1--227745665_jpeg_jpg.rf.07cc25aa58a493530b6b19976f04ac6b.jpg'),
 PosixPath('/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/train/images/uwg_g-986__fliph_jpg.rf.58fa59b2cd322fa207a339a23894ee31.jpg'),
 PosixPath('/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/train/images/20211007104805_jpg.rf.7e906a628f8d86fffc038e5d6c961202.jpg'),
 PosixPath('/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/train/images/uwg_g-443_jpg.rf.90da8fdbd63db58dfbcfff4ec2b97f05.jpg'),
 PosixPath('/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/train/images/uwg_g-896__fliph_jpg.rf.1ccdd44167e763dc88da85fb6c9f373d.jpg'),
 PosixP

In [36]:
#requirements
!pip install ultralytics
from ultralytics import YOLO

In [37]:
#check the data structure
!ls $path
!cat $path/'data.yaml'

data.yaml  test  train	valid
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 15
names: ['Mask', 'can', 'cellphone', 'electronics', 'gbottle', 'glove', 'metal', 'misc', 'net', 'pbag', 'pbottle', 'plastic', 'rod', 'sunglasses', 'tire']

roboflow:
  workspace: object-detect-dmjpt
  project: ocean_waste
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/object-detect-dmjpt/ocean_waste/dataset/1

In [38]:
#load the pretrained model
model=YOLO('yolo11s.pt')

In [39]:
#training the model
model.train(
    data=path/'data.yaml',
    epochs=150,
    imgsz=640,
    batch=16
)

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79fb9b3ad520>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,   

In [40]:
#finding the path of the best model

import glob
print(glob.glob("runs/detect/**/best.pt", recursive=True))

['runs/detect/train3/weights/best.pt', 'runs/detect/train4/weights/best.pt']


In [42]:
#testing the model

model = YOLO('runs/detect/train4/weights/best.pt')  #Loads the best trained weights

model.val(split='test')




Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,418,605 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.2±0.2 ms, read: 105.6±55.6 MB/s, size: 39.5 KB)
val: Scanning /kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/test/labels... 501 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 501/501 583.9it/s 0.9s
WARNING ⚠️ val: Cache directory /kaggle/input/underwater-plastic-pollution-detection/underwater_plastics/test is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 32/32 3.8it/s 8.4s
                   all        501        966      0.774      0.701      0.773      0.488
                  Mask         33         37          1       0.66      0.895      0.699
                   can         13         19      0.781      0.752      0.793      0.394
             cellphone         4

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79fcd869d070>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,   